In [ ]:
# ------------------TASK1-----------------
def levenshtein(s1, s2):
    m = len(s1)
    n = len(s2)

     # empty strings case
    if m == 0:
        return n
    if n == 0:
        return m

    # creating the matrix
    matrix = [[0 for _ in range(n+1)] for _ in range(m+1)]

    # first row and columnn
    for i in range(m+1):
        matrix[i][0] = i
    for j in range(n+1):
        matrix[0][j] = j

    # looping from secocnd column and row
    for i in range(1, m+1):
        for j in range(1, n+1):
            if s1[i - 1] == s2[j - 1]:  # same letters
                matrix[i][j] = matrix[i - 1][j - 1] # assign the diagonal value
            else:
                matrix[i][j] = 1 + min(matrix[i][j-1], matrix[i - 1][ j], matrix[i-1][j - 1]) # choosing min from tip, left, diagonal
 
    # backtracking to count operations
    i, j = m, n
    insertions = deletions = substitutions = 0
    while i > 0 or j > 0:
        if i > 0 and j > 0 and s1[i - 1] == s2[j - 1]:
            i -= 1
            j -= 1
        elif i > 0 and j > 0 and matrix[i][j] == matrix[i - 1][j - 1] + 1:
            substitutions += 1
            i -= 1
            j -= 1
        elif i > 0 and matrix[i][j] == matrix[i - 1][j] + 1:
            deletions += 1
            i -= 1
        elif j > 0 and matrix[i][j] == matrix[i][j - 1] + 1:
            insertions += 1
            j -= 1

    for row in matrix:
        print(row)
    
    print("\n\n")
    print(f"Levenshtein Distance: {matrix[m][n]}")
    print("\n")
    print(f"Insertions: {insertions}")
    print(f"Deletions: {deletions}")
    print(f"Substitutions: {substitutions}")


def main():
    str1 = "kitten"
    str2 = "sitting"

    levenshtein(str1, str2)
    


if __name__ == "__main__":
    main()

[0, 1, 2, 3, 4, 5, 6, 7]
[1, 1, 2, 3, 4, 5, 6, 7]
[2, 2, 1, 2, 3, 4, 5, 6]
[3, 3, 2, 1, 2, 3, 4, 5]
[4, 4, 3, 2, 1, 2, 3, 4]
[5, 5, 4, 3, 2, 2, 3, 4]
[6, 6, 5, 4, 3, 3, 2, 3]



Levenshtein Distance: 3


Insertions: 1
Deletions: 0
Substitutions: 2


In [ ]:
# ----------------------TASK2---------------------------

def levenshtein_words(ref_words, hyp_words):
    m, n = len(ref_words), len(hyp_words)

    if m == 0:
        return n, 0, n, 0
    if n == 0:
        return m, m, 0, 0

    matrix = [[0 for _ in range(n+1)] for _ in range(m+1)]

    for i in range(m+1):
        matrix[i][0] = i
    for j in range(n+1):
        matrix[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            if ref_words[i-1] == hyp_words[j-1]:    # comparing words
                matrix[i][j] = matrix[i-1][j-1]
            else:
                matrix[i][j] = 1 + min(
                    matrix[i][j-1],    
                    matrix[i-1][j],     
                    matrix[i-1][j-1]    
                )

    
    i, j = m, n
    insertions = deletions = substitutions = 0
    while i > 0 or j > 0:
        if i > 0 and j > 0 and ref_words[i-1] == hyp_words[j-1]:
            i -= 1
            j -= 1
        elif i > 0 and j > 0 and matrix[i][j] == matrix[i-1][j-1] + 1:
            substitutions += 1
            i -= 1
            j -= 1
        elif i > 0 and matrix[i][j] == matrix[i-1][j] + 1:
            deletions += 1
            i -= 1
        elif j > 0 and matrix[i][j] == matrix[i][j-1] + 1:
            insertions += 1
            j -= 1

    return matrix[m][n], insertions, deletions, substitutions


def main():
    # read files
    with open("d:\\MyCODES\\fall25\\ai\\lab1\\reference.txt", "r") as f:
        reference = f.read().strip().split()

    with open("d:\\MyCODES\\fall25\\ai\\lab1\\hypothesis.txt", "r") as f:
        hypothesis = f.read().strip().split()

    # compute word-level Levenshtein
    distance, ins, dele, subs = levenshtein_words(reference, hypothesis)

    # write to result.txt
    with open("result.txt", "w") as f:
        f.write(f"Levenshtein distance is {distance}\n")
        f.write(f"Insertions {ins}\n")
        f.write(f"Deletions {dele}\n")
        f.write(f"Substitutions {subs}\n")

    print("Result written to result.txt")


if __name__ == "__main__":
    main()


Result written to result.txt


In [ ]:
# ------------------------TASK3----------------------------

def levenshtein_words_ignore(ref_words, hyp_words, ignore_words):
    m, n = len(ref_words), len(hyp_words)

    # DP matrix
    matrix = [[0 for _ in range(n+1)] for _ in range(m+1)]
    for i in range(m+1):
        matrix[i][0] = i
    for j in range(n+1):
        matrix[0][j] = j

    # fill DP but also look for ignore word
    for i in range(1, m+1):
        for j in range(1, n+1):
            if ref_words[i-1] == hyp_words[j-1]:
                matrix[i][j] = matrix[i-1][j-1]
            else:
                cost_sub = 1
                if ref_words[i-1] in ignore_words and hyp_words[j-1] in ignore_words:
                    cost_sub = 0  # ignore substitution

                cost_del = 0 if ref_words[i-1] in ignore_words else 1
                cost_ins = 0 if hyp_words[j-1] in ignore_words else 1

                matrix[i][j] = min(
                    matrix[i-1][j] + cost_del,      # deletion # isntead of adding 1, we add the cost_sub which will be zero of the woeds are ignore words
                    matrix[i][j-1] + cost_ins,      # insertion
                    matrix[i-1][j-1] + cost_sub     # substitution
                )

    # backtrack with same cost rules
    i, j = m, n
    insertions = deletions = substitutions = 0

    while i > 0 or j > 0:
        if i > 0 and j > 0 and ref_words[i-1] == hyp_words[j-1]:
            i -= 1
            j -= 1
        else:
            # recompute costs for this cell
            cost_sub = 1
            if i > 0 and j > 0 and ref_words[i-1] in ignore_words and hyp_words[j-1] in ignore_words:
                cost_sub = 0

            cost_del = 1
            if i > 0 and ref_words[i-1] in ignore_words:
                cost_del = 0

            cost_ins = 1
            if j > 0 and hyp_words[j-1] in ignore_words:
                cost_ins = 0

            if i > 0 and j > 0 and matrix[i][j] == matrix[i-1][j-1] + cost_sub:
                if cost_sub == 1:
                    substitutions += 1
                i -= 1
                j -= 1
            elif i > 0 and matrix[i][j] == matrix[i-1][j] + cost_del:
                if cost_del == 1:
                    deletions += 1
                i -= 1
            elif j > 0 and matrix[i][j] == matrix[i][j-1] + cost_ins:
                if cost_ins == 1:
                    insertions += 1
                j -= 1
            else:
                # fallback (shouldn't hit often)
                i -= 1
                j -= 1

    return matrix[m][n], insertions, deletions, substitutions



def main():
    # read files
    with open("d:\\MyCODES\\fall25\\ai\\lab1\\reference.txt", "r") as f:
        reference = f.read().strip().split()

    with open("d:\\MyCODES\\fall25\\ai\\lab1\\hypothesis.txt", "r") as f:
        hypothesis = f.read().strip().split()

    ignore_words = {"the", "of", "and", "a", "be", "this", "there", "an", "been", "some"}

    distance, ins, dele, subs = levenshtein_words_ignore(reference, hypothesis, ignore_words)

    # write result2.txt
    with open("d:\\MyCODES\\fall25\\ai\\lab1\\result2.txt", "w") as f:
        f.write(f"Levenshtein distance is {distance}\n")
        f.write(f"Insertions {ins}\n")
        f.write(f"Deletions {dele}\n")
        f.write(f"Substitutions {subs}\n")

    print("Result written to result2.txt")


if __name__ == "__main__":
    main()


Result written to result2.txt ✅
